# 🧠 Primeros pasos con Kùzu

Este notebook es una guía práctica para explorar las capacidades de [Kùzu](https://kuzudb.com/), una base de datos de grafos de alto rendimiento, embebida y orientada a cargas analíticas y flujos de trabajo con machine learning basado en grafos.

Aquí encontrarás ejemplos prácticos que muestran cómo:

- Configurar e inicializar una base de datos Kùzu.
- Definir esquemas de nodos y relaciones.
- Cargar datos desde archivos CSV.
- Ejecutar consultas en Cypher para explorar patrones en el grafo.
- Procesar los resultados utilizando Python.

Los ejemplos de esta notebook utilizan el lenguaje de consulta **Cypher**.  
Si no estás familiarizado con Cypher, se recomienda revisar el siguiente tutorial oficial de Kùzu para entender su sintaxis y conceptos básicos:[Tutorial de Cypher en Kùzu](https://docs.kuzudb.com/tutorials/cypher/)

---

#### 📌 Nota

Este notebook es un **documento vivo**.  
Se irán agregando nuevos ejemplos y casos de uso avanzados de forma progresiva.

¡Siéntete libre de clonar, ejecutar y adaptar estos ejemplos a tus propios proyectos!

---

## Ejemplo 1

In [13]:
! wget -O data/sigue_a.csv "https://drive.usercontent.google.com/u/0/uc?id=1CeL2CpVhPH57SMzrxby6me2uAG9S4rqn&export=download"

--2025-07-08 09:58:32--  https://drive.usercontent.google.com/u/0/uc?id=1CeL2CpVhPH57SMzrxby6me2uAG9S4rqn&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.128.33, 2800:3f0:4002:804::2001
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.128.33|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.usercontent.google.com/uc?id=1CeL2CpVhPH57SMzrxby6me2uAG9S4rqn&export=download [following]
--2025-07-08 09:58:32--  https://drive.usercontent.google.com/uc?id=1CeL2CpVhPH57SMzrxby6me2uAG9S4rqn&export=download
Reusing existing connection to drive.usercontent.google.com:443.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1CeL2CpVhPH57SMzrxby6me2uAG9S4rqn&export=download [following]
--2025-07-08 09:58:32--  https://drive.usercontent.google.com/download?id=1CeL2CpVhPH57SMzrxby6me2uAG9S4rqn&export=download
Reusing ex

In [12]:
! wget -O data/usuarios.csv "https://drive.usercontent.google.com/u/0/uc?id=1h68ao8nqlszz0gT9-o5eSEv20lBN8aPS&export=download"

--2025-07-08 09:58:27--  https://drive.usercontent.google.com/u/0/uc?id=1h68ao8nqlszz0gT9-o5eSEv20lBN8aPS&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.128.33, 2800:3f0:4002:804::2001
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.128.33|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.usercontent.google.com/uc?id=1h68ao8nqlszz0gT9-o5eSEv20lBN8aPS&export=download [following]
--2025-07-08 09:58:29--  https://drive.usercontent.google.com/uc?id=1h68ao8nqlszz0gT9-o5eSEv20lBN8aPS&export=download
Reusing existing connection to drive.usercontent.google.com:443.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1h68ao8nqlszz0gT9-o5eSEv20lBN8aPS&export=download [following]
--2025-07-08 09:58:29--  https://drive.usercontent.google.com/download?id=1h68ao8nqlszz0gT9-o5eSEv20lBN8aPS&export=download
Reusing ex

In [1]:
import kuzu
import os
import shutil

In [2]:
# --- 1. Preparación: Limpiar y crear directorio para la base de datos ---

db_path = "kuzu_db"
if os.path.exists(db_path):
    shutil.rmtree(db_path)

In [3]:
# --- 2. Inicialización: Crear una base de datos en disco y una conexión ---
# Kùzu crea el directorio si no existe.

db = kuzu.Database(db_path)
conn = kuzu.Connection(db)
print(f"Base de datos Kùzu creada en: {db_path}")

Base de datos Kùzu creada en: kuzu_db


In [4]:
# --- 3. Definición del Esquema: Crear tablas de nodos y relaciones ---
# Kùzu utiliza un esquema estructurado, por lo que las tablas deben definirse primero.
# Se define una tabla de nodos 'Usuario' con un 'id' como clave primaria.

conn.execute("CREATE NODE TABLE Usuario(id INT64, nombre STRING, PRIMARY KEY (id))")

# Se define una tabla de relaciones 'SIGUE_A' que conecta nodos 'Usuario'.

conn.execute("CREATE REL TABLE SIGUE_A(FROM Usuario TO Usuario, desde_anio INT64)")
print("Esquema de 'Usuario' y 'SIGUE_A' creado.")

Esquema de 'Usuario' y 'SIGUE_A' creado.


In [14]:
# --- 4. Carga de Datos: Ingestar datos desde archivos CSV ---
# El comando COPY es altamente eficiente para la carga masiva de datos.
# Se asume que los CSV tienen cabeceras que coinciden con los nombres de las propiedades.

print("Cargando datos desde archivos CSV...")
conn.execute('COPY Usuario FROM "data/usuarios.csv" (HEADER=true)')
conn.execute('COPY SIGUE_A FROM "data/sigue_a.csv" (HEADER=true)')
print("Datos cargados correctamente.")

Cargando datos desde archivos CSV...
Datos cargados correctamente.


In [22]:
# --- 5. Consulta de Datos: Ejecutar una consulta Cypher para encontrar patrones ---
# Se busca un patrón: un usuario (u1) que sigue a otro (u2), que a su vez sigue a un tercero (u3).
# Esto es una consulta de "amigos de amigos" o de dos saltos.

query = """
    MATCH (u1:Usuario)-->(u2:Usuario)-->(u3:Usuario)
    WHERE u1.nombre = 'Usuario_2'
    RETURN u1.nombre AS seguidor, u2.nombre AS intermediario, u3.nombre AS seguido
"""
print(f"\nEjecutando consulta Cypher:\n{query}")
query_result = conn.execute(query)


Ejecutando consulta Cypher:

    MATCH (u1:Usuario)-->(u2:Usuario)-->(u3:Usuario)
    WHERE u1.nombre = 'Usuario_2'
    RETURN u1.nombre AS seguidor, u2.nombre AS intermediario, u3.nombre AS seguido



In [23]:
# --- 6. Procesamiento de Resultados: Iterar y mostrar los resultados ---

print("\nResultados de la consulta:")
while query_result.has_next():
    row = query_result.get_next()
    # row es una lista de valores en el orden especificado en RETURN
    print(f"{row[0]} sigue a {row[1]} a través de {row[2]}")

# El cierre de la conexión y la base de datos no es estrictamente necesario
# en scripts simples, pero es una buena práctica en aplicaciones más grandes.


Resultados de la consulta:
Usuario_2 sigue a Usuario_27 a través de Usuario_7
Usuario_2 sigue a Usuario_27 a través de Usuario_38
Usuario_2 sigue a Usuario_27 a través de Usuario_48


## Ejemplo 2

In [24]:
! curl -o data/tutorial_data.zip https://rgw.cs.uwaterloo.ca/kuzu-test/tutorial/tutorial_data.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2559  100  2559    0     0   1933      0  0:00:01  0:00:01 --:--:--  1934


In [28]:
! unzip ./data/tutorial_data.zip -d ./data

Archive:  ./data/tutorial_data.zip
   creating: ./data/tutorial_data/node/
  inflating: ./data/tutorial_data/node/post.csv  
  inflating: ./data/tutorial_data/node/user.csv  
   creating: ./data/tutorial_data/relation/
  inflating: ./data/tutorial_data/relation/FOLLOWS.csv  
  inflating: ./data/tutorial_data/relation/LIKES.csv  
  inflating: ./data/tutorial_data/relation/POSTS.csv  


In [29]:
! rm ./data/tutorial_data.zip

In [45]:
db_path = "kuzu_db"
if os.path.exists(db_path):
    shutil.rmtree(db_path)

db = kuzu.Database(db_path)
conn = kuzu.Connection(db)
print(f"Base de datos Kùzu creada en: {db_path}")

Base de datos Kùzu creada en: kuzu_db


In [46]:
conn.execute("""
    CREATE NODE TABLE User (
        user_id INT64 PRIMARY KEY,
        username STRING,
        account_creation_date DATE
    )""")

conn.execute("""
    CREATE NODE TABLE Post (
        post_id INT64 PRIMARY KEY,
        post_date DATE,
        like_count INT64,
        retweet_count INT64
    )""")

conn.execute("""
    CREATE REL TABLE FOLLOWS (
        FROM User TO User
    )""")

conn.execute("""
    CREATE REL TABLE POSTS (
        FROM User TO Post
    )""")

conn.execute("""
    CREATE REL TABLE LIKES (
        FROM User TO Post
    )""")

print("Esquema de 'User', 'FOLLOWS', 'POSTED' y 'LIKES' creado.")

Esquema de 'User', 'FOLLOWS', 'POSTED' y 'LIKES' creado.


In [47]:
conn.execute("COPY User FROM './data/tutorial_data/node/user.csv'")
print("Datos cargados correctamente.")

Datos cargados correctamente.


In [48]:
conn.execute("COPY Post FROM './data/tutorial_data/node/post.csv'")
print("Datos cargados correctamente.")

Datos cargados correctamente.


In [49]:
conn.execute("COPY FOLLOWS FROM './data/tutorial_data/relation/FOLLOWS.csv'")
print("Datos cargados correctamente.")

Datos cargados correctamente.


In [50]:
conn.execute("COPY POSTS FROM './data/tutorial_data/relation/POSTS.csv'")
print("Datos cargados correctamente.")

Datos cargados correctamente.


In [51]:
conn.execute("COPY LIKES FROM './data/tutorial_data/relation/LIKES.csv'")
print("Datos cargados correctamente.")

Datos cargados correctamente.


In [52]:
## Mostrar información de la tabla
result = conn.execute("CALL SHOW_TABLES() RETURN *")

print(result.get_column_names())
while result.has_next():
    print(result.get_next())

['id', 'name', 'type', 'database name', 'comment']
[0, 'User', 'NODE', 'local(kuzu)', '']
[2, 'FOLLOWS', 'REL', 'local(kuzu)', '']
[1, 'Post', 'NODE', 'local(kuzu)', '']
[3, 'POSTS', 'REL', 'local(kuzu)', '']
[4, 'LIKES', 'REL', 'local(kuzu)', '']


In [61]:
## ¿Qué usuario tiene más seguidores? ¿Y cuántos seguidores tiene?
result = conn.execute("""MATCH (u1:User)-[f:FOLLOWS]->(u2:User)
                        RETURN u2.username
                        LIMIT 5""")

print(result.get_column_names())
while result.has_next():
    print(result.get_next())

print("\n")
print("-" * 100)
print("\n")

result = conn.execute("""MATCH (u1:User)-[f:FOLLOWS]->(u2:User)
                        RETURN u2.username, COUNT(u2) AS follower_count
                        LIMIT 5""")

print(result.get_column_names())
while result.has_next():
    print(result.get_next())

print("\n")
print("-" * 100)
print("\n")

result = conn.execute("""MATCH (u1:User)-[f:FOLLOWS]->(u2:User)
                        RETURN u2.username, COUNT(u2) AS follower_count
                        ORDER BY follower_count DESC
                        LIMIT 1""")

print(result.get_column_names())
while result.has_next():
    print(result.get_next())

print("\n")
print("-" * 100)
print("\n")

result = conn.execute("""MATCH (u1:User)-[f:FOLLOWS]->(u2:User)
                        WITH u2, COUNT(u1) as follower_count
                        WITH MAX(follower_count) as max_count
                        MATCH (u1:User)-[f:FOLLOWS]->(u2:User)
                        WITH u2, COUNT(u1) as follower_count, max_count
                        WHERE follower_count = max_count
                        RETURN u2.username, follower_count""")

print(result.get_column_names())
while result.has_next():
    print(result.get_next())




['u2.username']
['coolwolf752']
['stormfox762']
['stormninja678']
['darkdog878']
['brightninja683']


----------------------------------------------------------------------------------------------------


['u2.username', 'follower_count']
['stormcat597', 2]
['epiccat105', 4]
['fastgirl798', 4]
['darkdog878', 6]
['epicking81', 3]


----------------------------------------------------------------------------------------------------


['u2.username', 'follower_count']
['darkdog878', 6]


----------------------------------------------------------------------------------------------------


['u2.username', 'follower_count']
['stormninja678', 6]
['darkdog878', 6]


## Ejemplo 3

In [86]:
! curl -L -o data/user_data/follows.csv https://raw.githubusercontent.com/kuzudb/kuzu/refs/heads/master/dataset/demo-db/csv/follows.csv
! curl -L -o data/user_data/lives-in.csv https://raw.githubusercontent.com/kuzudb/kuzu/refs/heads/master/dataset/demo-db/csv/lives-in.csv
! curl -L -o data/user_data/user.csv https://raw.githubusercontent.com/kuzudb/kuzu/refs/heads/master/dataset/demo-db/csv/user.csv
! curl -L -o data/user_data/city.csv https://raw.githubusercontent.com/kuzudb/kuzu/refs/heads/master/dataset/demo-db/csv/city.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    70  100    70    0     0    444      0 --:--:-- --:--:-- --:--:--   445
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    60  100    60    0     0    469      0 --:--:-- --:--:-- --:--:--   472
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    37  100    37    0     0    166      0 --:--:-- --:--:-- --:--:--   166
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    46  100    46    0     0    105      0 --:--:-- --:--:-- --:--:--   105


In [91]:
def main():
    # Crear una base de datos en memoria (no persistente en disco)
    db = kuzu.Database(":memory:")
    
    # Establecer una conexión con la base de datos
    conn = kuzu.Connection(db)

    # Crear tabla de nodos 'User' con nombre como clave primaria
    conn.execute("CREATE NODE TABLE User(name STRING, age INT64, PRIMARY KEY (name))")
    
    # Crear tabla de nodos 'City' con nombre como clave primaria
    conn.execute("CREATE NODE TABLE City(name STRING, population INT64, PRIMARY KEY (name))")
    
    # Crear tabla de relaciones 'Follows' entre usuarios, con un atributo 'since'
    conn.execute("CREATE REL TABLE Follows(FROM User TO User, since INT64)")
    
    # Crear tabla de relaciones 'LivesIn' que conecta un usuario con una ciudad
    conn.execute("CREATE REL TABLE LivesIn(FROM User TO City)")

    # Cargar datos desde archivos CSV en sus respectivas tablas
    conn.execute('COPY User FROM "./data/user_data/user.csv"')
    conn.execute('COPY City FROM "./data/user_data/city.csv"')
    conn.execute('COPY Follows FROM "./data/user_data/follows.csv"')
    conn.execute('COPY LivesIn FROM "./data/user_data/lives-in.csv"')

    # Ejecutar una consulta Cypher para obtener las relaciones de seguimiento entre usuarios
    response = conn.execute(
        """
        MATCH (a:User)-[f:Follows]->(b:User)
        RETURN a.name, b.name, f.since;
        """
    )
    
    # Iterar sobre los resultados y mostrarlos en consola
    while response.has_next():
        print(response.get_next())

In [92]:
main()

['Adam', 'Karissa', 2020]
['Adam', 'Zhang', 2020]
['Karissa', 'Zhang', 2021]
['Zhang', 'Noura', 2022]


## Ejemplo 4

In [94]:
! curl -L -o data/user_movie/movies.csv https://kuzudb.com/data/movie-lens/movies.csv
! curl -L -o data/user_movie/users.csv https://kuzudb.com/data/movie-lens/users.csv
! curl -L -o data/user_movie/ratings.csv https://kuzudb.com/data/movie-lens/ratings.csv
! curl -L -o data/user_movie/tags.csv https://kuzudb.com/data/movie-lens/tags.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  520k    0  520k    0     0   429k      0 --:--:--  0:00:01 --:--:--  429k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2338    0  2338    0     0   3976      0 --:--:-- --:--:-- --:--:--  3982
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2327k    0 2327k    0     0  3286k      0 --:--:-- --:--:-- --:--:-- 3286k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  112k    0  112k    0     0   205k      0 --:--:-- --:--:-- --:--:--  205k


In [96]:
import shutil
import kuzu
db_path = './kuzu_db'
shutil.rmtree(db_path, ignore_errors=True)

def load_data(connection):
    connection.execute('CREATE NODE TABLE Movie (movieId INT64, year INT64, title STRING, genres STRING, PRIMARY KEY (movieId))')
    connection.execute('CREATE NODE TABLE User (userId INT64, PRIMARY KEY (userId))')
    connection.execute('CREATE REL TABLE Rating (FROM User TO Movie, rating DOUBLE, timestamp INT64)')
    connection.execute('CREATE REL TABLE Tags (FROM User TO Movie, tag STRING, timestamp INT64)')

    connection.execute('COPY Movie FROM "data/user_movie/movies.csv" (HEADER=TRUE)')
    connection.execute('COPY User FROM "data/user_movie/users.csv" (HEADER=TRUE)')
    connection.execute('COPY Rating FROM "data/user_movie/ratings.csv" (HEADER=TRUE)')
    connection.execute('COPY Tags FROM "data/user_movie/tags.csv" (HEADER=TRUE)')

db = kuzu.Database(db_path)
conn = kuzu.Connection(db)
load_data(conn)

In [102]:
# Ejecuta una consulta Cypher que selecciona hasta 5 películas de la tabla Movie.
# Solo se retorna el título (m.title) de cada película.
res1 = conn.execute(
    """
    MATCH (m:Movie)           
    RETURN m.title            
    LIMIT 10
    """
)

# Convierte el resultado de la consulta a un DataFrame de pandas y lo imprime en consola.
df = res1.get_as_df()
df.head()

,m.title
0,Toy Story (1995)
1,Jumanji (1995)
2,Grumpier Old Men (1995)
3,Waiting to Exhale (1995)
4,Father of the Bride Part II (1995)
